In [120]:
from IPython.display import display
import geopandas as gdf
import pandas as pd

shapes = gdf.read_file('./data/cb_2018_27_cousub_500k.zip')
# twin_cities_area = pd.read_csv('./data/twin_cities_neighborhoods.csv')
# I think the twin_cities_neighborhoods isn't a good file. It's missing stuff and includes things it shouldn't.

"""
What do I want to do.
1. Calculate the distance of all of them in relation to the airport and Oronoco.
2. Verify the miles calculation by spot checking a few with Google Maps
3. Find a way to overlay information (e.g. distance) on the shapes or popup.
3. Pull in other metadata (e.g. real estate, crime, water, etc...)
"""

# The city shapes are provided in a geographic CRS NAD83 (authority code: EPSG:4269). 
# A geographic projection is one in which that the coordinates are in latitude and longitude.
# The US Census Bureau uses NAD83 for most of their stuff. This is an un-projected datum.
# https://source.opennews.org/articles/choosing-right-map-projection/

# A projected CRS is in meters, feet, kilometers etc.
# Can see the active projection with: shapes.crs

# This isn't good for doing distance calculations
# Re-projecting to EPSG:4087, WGS 84 / World Equidistant Cylindrical
# This projection's UOM is in meters.
# Note the popular online maps systems use EPSG:3857
# https://epsg.io/3857
# https://en.wikipedia.org/wiki/Web_Mercator_projection
# It may work out to project to EPSG:4087 for calculations 
# and then project to EPSG:3857 for visualizations.
shapes = shapes.to_crs("EPSG:4087")

# Find the centroid of all the shapes.
# Centroid vs representative_point
shapes['centroid'] = shapes.geometry.centroid


# The airport in in Fort Snelling
airport = shapes.loc[shapes['NAME'] == 'Fort Snelling']
airport_point = airport.iat[0, 11] # Get the centroid of the airport.

# Find the centroid of Oronoco
oronoco = shapes[(shapes['NAME'] == 'Oronoco') & (shapes['COUSUBFP'] == '48598') ]
oronoco_point = oronoco.iat[0, 11]


# Find the distanace of all the shapes to the airport.
# The distance calculation is done in meters which is then converted to miles.
# 1 meter = 0.000621371
# Divide # meters by 1609
METERS_TO_MILES = 1609
shapes['miles_to_ap'] = shapes['centroid'].distance(airport_point)/METERS_TO_MILES
shapes['miles_to_oronoco'] = shapes['centroid'].distance(oronoco_point)/METERS_TO_MILES

# display(shapes)

# shapes = pd.merge(shapes, twin_cities_area, left_on='NAME', right_on='City', how='inner')

# Project to the same CSR as google maps for visualization
shapes = shapes.to_crs("EPSG:3857")

MAX_DISTANCE_TO_AP = 15
close_to_ap = shapes[(shapes['miles_to_ap'] < MAX_DISTANCE_TO_AP) | ((shapes['NAME'] == 'Oronoco') & (shapes['COUSUBFP'] == '48598'))]
avg_distance_to_oronoco = round(close_to_ap['miles_to_oronoco'].mean(), 1)
print (f'Filtering by Maximum Distance to Airport: {MAX_DISTANCE_TO_AP}')
print(f'Viewing {len(close_to_ap.index)} neighborhoods.')
print(f'Avg Distance to Oronoco: {avg_distance_to_oronoco} miles')


# Create an interactive map using the folium library.
close_to_ap.explore('miles_to_ap', legend=True)


Filtering by Maximum Distance to Airport: 15
Viewing 37 neighborhoods.
Avg Distance to Oronoco: 69.4 miles
